In [2]:
import pandas as pd
import numpy as np

data_new = pd.read_csv('data_new.csv')

data_new.head()

data_new['secucategory'].value_counts()

"""计算前几天的收益"""
def stack2dataframe(df,index_nm,column_nm,feature_nm):
    """privot table"""
    return pd.pivot_table(
        df,
        index = index_nm,
        columns = column_nm,
        values = feature_nm).sort_index()

data_new['tradingday'] = data_new['tradingday'].astype(str)
data_new['secucode'] = data_new['secucode'].apply(lambda x:str(x).zfill(6))

data_new['pctChg'] = data_new['pctChg']/100

'''当天收益'''
pctChg = stack2dataframe(data_new,'tradingday','secucode','pctChg')

'''转成未来一天的收益！！！！！注意'''
changepct = pctChg.shift(-1)
df_changpct = changepct.stack().reset_index().rename(columns = {0:'changepct'})

# df_changpct.dtypes
data_new = pd.merge(data_new,df_changpct,on = ['tradingday','secucode'],how = 'left')

data_new['tradingday'].min(),data_new['tradingday'].max()

data_new['listedsector'] = data_new['secucode'].apply(lambda x: 1 if x[:3]=='300' else 0)
# data_new.rename(columns = {'pctChg':'changepct'},inplace = True)

data_new['date_dt'] = data_new.apply(
    lambda x:1 if x['changepct']<=-0.098 and x['tradingday']<'20200824'
              else 1 if  x['changepct']<=-0.198 and x['tradingday']>='20200824' and x['listedsector']==1
              else 1 if  x['changepct']<=-0.098 and x['tradingday']>='20200824' and x['listedsector']!=1
              else 0 ,axis=1)

data_new[(data_new['date_dt']==1)&(data_new['tradingday']<'20200824')]['changepct'].max()

data_new[(data_new['date_dt']==1)&(data_new['listedsector']==1)&(data_new['tradingday']>='20200824')]['changepct'].max()

'''计算label'''
def maxdrawdown(x):
    x = x.values
    return np.nanmax([np.nanmax((x[i]-x[i+1:])/x[i]) for i in range(len(x)-1)])

def riskjudge0(x):
    return 1 if x>=0.4 else 0

def riskjudge1(x):
    x= x.values
    return 1 if np.nansum(x)>=3 else 0

def riskjudge2(x):
    x= x.values
    return 1 if np.nansum(x[:-1]*x[1:])>=1 else 0

def riskjudge(x):
    return  riskjudge1(x) or riskjudge2(x)

data_new = data_new.sort_values(['secucode','tradingday'])

data_new['maxdrawdown'] = data_new.groupby(['secucode'])['changepct'].shift(-10).rolling(10).apply(
    lambda x:maxdrawdown(np.cumprod(x+1))).values


data_new['maxdrawdown_flag'] = data_new['maxdrawdown'].apply(lambda x:riskjudge0(x))
data_new['dt_flag'] = data_new.groupby(['secucode'])['date_dt'].shift(-10).rolling(10).apply(lambda x:riskjudge(x)).values
data_new['label'] = data_new.apply(lambda x:x['maxdrawdown_flag'] or x['dt_flag'],axis =1)
data_new['label'].value_counts()

data_new[(data_new['tradingday']>='20200101')&(data_new['tradingday']<='20201231')]['label'].value_counts()

'''导入特征'''
import os
file_list = [col for col in os.listdir('F:/南大实习') if 'df_res' in col ]
df_train = pd.read_csv('F:/南大实习/'+file_list[0])

df_train.dtypes
df_train.isnull().sum()
isnull_dict = dict(df_train.isnull().sum()/df_train.shape[0])
feature_columns = [col for col in isnull_dict.keys() if isnull_dict[col]<=0.1 and col not in ['tradingday','secucode']]

len(feature_columns)

df_train['tradingday'] = df_train['tradingday'].astype(str)
df_train['secucode'] = df_train['secucode'].apply(lambda x:str(x).zfill(6))

df_train = df_train.sort_values(['secucode'])

def Filterby3Sigma(x):
    x_ = np.array(x)
    std_ = np.nanstd(x_)
    mean_= np.nanmean(x_)
    low_ = mean_ - 3*std_
    up_ = mean_ + 3*std_
    x_[x_<low_] = low_
    x_[x_>up_] = up_
    return x_

def MinMaxScale(x):
    x = np.array(x)
    min_ = np.nanmin(x)
    max_ = np.nanmax(x)
    return (x-min_)/(max_ - min_) if max_ != min_ else x

def ZScoreScale(x):
    x = np.array(x)
    mean_ = np.nanmean(x)
    std_ = np.nanmax(x)
    return (x-mean_)/(std_) if std_!=0 else x

'''特征处理'''
'''异常值截断'''
for c in feature_columns:
    df_train[c] = df_train.groupby(['secucode'])[c].transform(Filterby3Sigma)

'''缺失值填充'''
fill_cols = [c for c in feature_columns if isnull_dict[c]>0]
for c in fill_cols:
    df_train[c].fillna(df_train['secucode'].map(df_train.groupby(['secucode'])[c].mean().to_dict()),inplace = True)
'''z4score归一化'''
for c in feature_columns:
    df_train[c] = df_train.groupby(['secucode'])[c].transform(ZScoreScale)

# df_train.dtypes
# df_train.head()
# df_train['tradingday'].min()
# df_train['tradingday'].max()


df_label = data_new[(data_new['tradingday']>='20201231')].copy()

'''构建最近股价波动特征：3天收益均值，5天收入均值，7天收入均值，7天收入波动等'''
df_label = df_label.sort_values(['secucode','tradingday'])
for N in [3,5,7,14,31]:
    df_label['pctChg%d_avg'%N] = df_label.groupby(['secucode']).rolling(N)['pctChg'].mean().values
    df_label['pctChg%d_std'%N] = df_label.groupby(['secucode']).rolling(N)['pctChg'].std().values

start_date = df_label.dropna(subset=['pctChg31_avg'])['tradingday'].min()
df_label = df_label[df_label['tradingday']>=start_date].copy()

'''查看样本分布'''
df_label[df_label['tradingday']<='20211231']['label'].value_counts()

'''扩充样本'''
df_label['label1'] = df_label.groupby(['secucode']).rolling(30)['label'].sum().values
df_label['label1'] = df_label['label1'].apply(lambda x:1 if x>=1 else 0)
df_label[df_label['tradingday']<='20211231']['label1'].value_counts()

'''合并label和features'''
# train_data = 
df_label = pd.merge(df_label ,df_train,on = ['tradingday','secucode'],how = 'left')
feature_columns = feature_columns+[c for c in df_label.columns if 'pctChg' in c]

# feature_columns

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn import metrics
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score



'''样本抽样'''
train_start_date = start_date
train_end_date = '20211231'
test_start_date = '20220101'
test_end_date = df_train['tradingday'].max()

train_data = df_label[(df_label['tradingday']>=train_start_date)&(df_label['tradingday']<=train_end_date)].copy()
test_data = df_label[(df_label['tradingday']>=test_start_date)&(df_label['tradingday']<=test_end_date)].copy()

df_negative = shuffle(train_data[train_data['label']==0])
df_positive = shuffle(train_data[train_data['label']==1])

negative_cnt = df_positive.shape[0]*100
df_negative_sample = df_negative.sample(negative_cnt)
df_trains = shuffle(pd.concat([df_positive,df_negative_sample]))

df_trains.shape

def train_model(train,usefulcols,label_col,params):
    train_ = train.dropna(subset = [label_col],how= 'any')
    print(train_.shape)
    training,testing = train_test_split(train_,test_size=0.2,random_state=2019)
    lgb_train = lgb.Dataset(training[usefulcols],
                            label=list(training[label_col]),
                            #weight=training['weight'],
                            categorical_feature=[],
                            free_raw_data=False)
    lgb_valid = lgb.Dataset(testing[usefulcols],
                            label=list(testing[label_col]),
                            #weight=testing['weight'],
                            categorical_feature=[],
                            reference=lgb_train)
    print("Begin training Model!")
    clf = lgb.train(params, 
                    lgb_train, 
                    valid_sets = [lgb_train,lgb_valid],
                    verbose_eval = 20, 
                    early_stopping_rounds = 10
                   )
    return clf


"""new params"""
binary_params = {
    'boosting_type':'gbdt',
    'n_estimators':1000,
    'learning_rate':0.05,
    'max_depth':5,
    'bagging_fraction':1,
    'feature_fraction':1,
    'min_sum_hessian_in_leaf':0.2,
    'lambda_l1':1,
    'lambda_l2':1,
    'metric':'auc'
}





binary_clf = train_model(
        df_trains,feature_columns,'label1',binary_params)

"""train1 model pred"""
binary_y_prob = binary_clf.predict(
    test_data[feature_columns], 
    num_iteration=binary_clf.best_iteration)
test_data['score'] = binary_y_prob

test_data.dropna(subset=['label1','label'],inplace= True)

for label_col in ['label','label1']:
    fpr, tpr, _ = metrics.roc_curve(test_data[label_col], test_data['score'], pos_label=1)
    roc_auc = metrics.auc(fpr, tpr)
    # 计算recall、precision、average_precision
    precision, recall, _ = precision_recall_curve(test_data[label_col], test_data['score'])
    average_precision = average_precision_score(test_data[label_col], test_data['score'])
    print(label_col,roc_auc)

import matplotlib.pylab as plt


plt.figure(dpi=300,figsize=(24,8))
# plt.figure(figsize=[100,50],dpi=100)
lgb.plot_importance(binary_clf, max_num_features=10,importance_type='gain')
plt.title("Featurertances")
plt.show()

# import matplotlib.pyplot as plt

from sklearn.metrics import roc_auc_score, roc_curve


gbm_auc = roc_auc_score(test_data['label'].values, test_data['score'].values)  # 计算auc
gbm_fpr, gbm_tpr, gbm_threasholds = roc_curve(test_data['label'].values, test_data['score'].values)  # 计算ROC的值
plt.title("roc_curve of %s(AUC=%.4f)" % ('gbm', gbm_auc))
plt.xlabel('Specificity')  # specificity = 1 - np.array(gbm_fpr))
plt.ylabel('Sensitivity')  # sensitivity = gbm_tpr
plt.plot(list(1 - np.array(gbm_fpr)), gbm_tpr)
plt.gca().invert_xaxis()  # 将X轴反转
plt.show()


gbm_auc = roc_auc_score(test_data['label1'].values, test_data['score'].values)  # 计算auc
gbm_fpr, gbm_tpr, gbm_threasholds = roc_curve(test_data['label1'].values, test_data['score'].values)  # 计算ROC的值
plt.title("roc_curve of %s(AUC=%.4f)" % ('gbm', gbm_auc))
plt.xlabel('Specificity')  # specificity = 1 - np.array(gbm_fpr))
plt.ylabel('Sensitivity')  # sensitivity = gbm_tpr
plt.plot(list(1 - np.array(gbm_fpr)), gbm_tpr)
plt.gca().invert_xaxis()  # 将X轴反转
plt.show()

test_data.shape

test_data['tradingday'].max()

'''保存数据和模型'''

df_trains.to_csv('df_trains.csv',index = None)
test_data.to_csv('test_data.csv',index = None)
binary_clf.save_model('gjycbd_model_2021.txt')

test_data.head()

test_data[test_data['label']==1]['score'].mean()

np.nanpercentile(test_data['score'],[10,20,30,40,50,60,70,80,90,95,97])

np.nanpercentile(test_data['score'],97)

test_data[(test_data['score']>=np.nanpercentile(test_data['score'],97))&]

test_secucode = ['000736','002147','002306','002338','000068','002684','000633',
'002106','002569','002506','000152','000670','000972','002865',
'002911','002175']
test_data[test_data['secucode'].isin(test_secucode)]['label'].unique()

df_check =pd.read_excel('主板.xlsx')

df_check =pd.read_excel('主板.xlsx')
df_check.head()

true_secucdoe = [str(c).zfill(6) for c in df_check[df_check['函件类别']=='关注函']['公司代码'].unique()]

len(test_data[(test_data['secucode'].isin(true_secucdoe))&
          (test_data['score']>=np.nanpercentile(df_trains['score'] ,
                85))]['secucode'].unique())/len(true_secucdoe)

np.nanpercentile(test_data['score'],[10,20,30,40,50,60,70,80,90,95,97])

np.nanpercentile(test_data['score'],[10,20,30,40,50,60,70,80,90,95,97])

np.nanpercentile(test_data[(test_data['secucode'].isin(true_secucdoe))&(test_data['label']==1)]['score'],
                [10,20,30,40,50,60,70,80,90])

test_data[(test_data['secucode'].isin(true_secucdoe))&(test_data['label']==1)][['tradingday','secucode','label','score']]

test_data[(test_data['secucode'].isin(true_secucdoe))&(test_data['label']==1)][['tradingday','secucode','label','score']]['secucode'].unique()

df_check[df_check['公司代码']=='000620']

df_check['公司代码'] = df_check['公司代码'].apply(lambda x:str(x).zfill(6))

np.nanpercentile(test_data[(test_data['secucode'].isin(true_secucdoe))&(test_data['label']==0)]['score'],
                [10,20,30,40,50,60,70,80,90])

check_list = ['301025','300202','000953','600856','600444','002178','002640','002059','300181',
'002473','002256','600976','603297','600272','000525','603536','600734','300199',
'600671','002207','002313','600190','600358','002200','002499','301060','300306',
'002366','002620','603333','002053','600766','600818','603602','001234','001206',
'002779','002083','603214','300279','688171','603912','001267','002530','603138',
'603825','600291','600896','000606','600232','688075','002828','300672','601595',
'002062','002811','000017','300157','601880','603577','002546','002537','300084',
'002865','002908','000978','603150','002905','600365','002487','600149','603858',
'000892','603811','002589','002856','601700','603197','603108','603183','688221',
'603199','000779','300683','000707','300439','000007','000546','300056','300612',
'002566','000619','300942','600338','605167','300483','603168','002564','605378',
'002766','000504','002813','688102','300560','002166','000710','000752','603219',
'002194','603188','301118','603230','600566','600652','002694','002629','601107',
'300325','300213','300248','601865','300818','605033','600992','600129','603096',
'002232','603555','301166','002872','002639','000673','002621','600833','300588',
'002776','603322','603078','603636','603716','600396','603132','688105','603335',
'000506','600490','002427','300703','300966','600629']

check_list = [c for c in check_list if c[:3] in ['002','001','003']]

test_data[test_data['score']]

binary_y_probs = binary_clf.predict(
    df_trains[feature_columns], 
    num_iteration=binary_clf.best_iteration)
df_trains['score'] = binary_y_probs

true_secucdoe

len(test_data[(test_data['secucode'].isin(true_secucdoe))&
          (test_data['score']>=np.nanpercentile(df_trains['score'] ,
                85))]['secucode'].unique())/len(true_secucdoe)

len(check_list)

check_list

